In [ ]:
from init_notebook import *

In [ ]:
def plot_dynamics(
        dynamics, 
        pop_range: Tuple[float, float] = (0., 2.),
        count: int = 100,
        steps: int = 10,
        delta: float = 0.01,
        seed: int = 23,
        opacity: float = 0.1,
):
    rng = random.Random(seed)
    rows = []
    with tqdm(total=count*count) as progress:
        for y in range(count):
            for x in range(count):
                progress.update()
                populations = dynamics.init(*(
                    #rng.uniform(*pop_range) if j < 2 else 1
                    (x if j == 0 else y) / count * (pop_range[1] - pop_range[0]) + pop_range[0] + rng.uniform(-0.001, 0.001)
                    for j in range(2)
                ))
                for step_i in range(steps+1):
                    new_populations = dynamics(populations)
                    speed = math.sqrt(sum(math.pow(p - np, 2) for p, np in zip(populations, new_populations)))
                    populations = tuple(
                        populations[j] * (1. - delta) + delta * new_pop
                        for j, new_pop in enumerate(new_populations)
                    )
                    rows.append({
                        **{chr(96+(j+24)%26): p for j, p in enumerate(populations)},
                        "speed": speed,
                        "step": steps - step_i,
                    })
                
    df = pd.DataFrame(rows)
    display(
        px.scatter(
            df,
            x="x", y="y",
            #color="speed",
            height=1200,
            hover_data=["speed"],
            opacity=opacity,
            size="step",
        )
    )

class Dynamics:
    def __init__(
            self,
    ):
        self.num_populations = 3

    def init(self, x: float, y: float):
        return x, y, 2.
        
    def __call__(
            self,
            populations: Tuple[float, ...],
    ):
        # rabbits, foxes, grass
        r, f, g = populations
        rn = r*g - f*r        
        fn = f*max(0, r - g/2)               
        gn = g - r*g          
        return rn, fn, gn

plot_dynamics(
    Dynamics(), count=40, opacity=.5, delta=0.001,
    #pop_range=(0, .5),
)
        

In [ ]:
def plot_dynamics(
        dynamics, 
        pop_range: Tuple[float, float] = (0., 2.),
        count: int = 100,
        seed: int = 23,
):
    rng = random.Random(seed)
    rows = []
    with tqdm(total=count*count) as progress:
        screen_scale = count / (pop_range[1] - pop_range[0])
        for y in range(count):
            for x in range(count):
                progress.update()
                populations = dynamics.init(*(
                    #rng.uniform(*pop_range) if j < 2 else 1
                    (x if j == 0 else y) / count * (pop_range[1] - pop_range[0]) + pop_range[0] #+ rng.uniform(-0.001, 0.001)
                    for j in range(2)
                ))
                new_populations = dynamics(populations)
                distance = math.sqrt(sum(math.pow(p - np, 2) for p, np in zip(populations, new_populations)))
                grad = tuple((p - np) / max(distance, 10e-5) for p, np in zip(populations, new_populations))
                for sign in (-1, 1, None):
                    rows.append({
                        **({
                            chr(96+(j+24)%26): (
                                (p + sign * g / screen_scale) if sign is not None else None
                            )
                            for j, (p, g) in enumerate(zip(populations, grad))
                        }),
                        "distance": distance,
                        #"color": populations[2],
                    })
                
    df = pd.DataFrame(rows)
    display(
        px.line(
            df,
            x="x", y="y",
            height=1200,
            hover_data=["distance"],
        )
    )

class Dynamics:
    def __init__(
            self,
    ):
        self.num_populations = 3

    def init(self, x: float, y: float):
        return x, y, 2
        
    def __call__(
            self,
            populations: Tuple[float, ...],
    ):
        # rabbits, foxes, grass
        r, f, g = populations
        fox_eat = f*max(0, r - g/2)
        rn = r*g - fox_eat
        fn = fox_eat              
        gn = g*(1+f*r) - r*g          
        return rn, fn, gn

plot_dynamics(
    Dynamics(), count=50, 
    pop_range=(0, 10),
)
        